# Dashboard with Plotly Dash

In [9]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

app = dash.Dash(__name__)
server = app.server

uniquelaunchsites = spacex_df['Launch Site'].unique().tolist()
lsites = []
lsites.append({'label': 'All Sites', 'value': 'All Sites'})
for site in uniquelaunchsites:
 lsites.append({'label': site, 'value': site})

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    dcc.Dropdown(id='site_dropdown',options=lsites,placeholder='Select a Launch Site here', searchable = True , value = 'All Sites'),
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload_slider',
        min=0,
        max=10000,
        step=1000,
        marks = {0: '0 kg', 1000: '1000 kg', 2000: '2000 kg', 3000: '3000 kg', 4000: '4000 kg', 5000: '5000 kg', 6000: '6000 kg', 7000: '7000 kg', 8000: '8000 kg', 9000: '9000 kg', 10000: '10000 kg'},
        value=[min_payload,max_payload]
    ),
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    ])

@app.callback(
     Output(component_id='success-pie-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value')]
)
def update_graph(site_dropdown):
    if (site_dropdown == 'All Sites'):
        filtered_df  = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(filtered_df, names = 'Launch Site', title = 'Total Success Launches By all sites')
    else:
        filtered_df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(filtered_df, names = 'class', title = 'Total Success Launches for site '+site_dropdown)
    return fig

@app.callback(
     Output(component_id='success-payload-scatter-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value'),Input(component_id="payload_slider", component_property="value")]
)
def update_scattergraph(site_dropdown,payload_slider):
    if site_dropdown == 'All Sites':
        lowest, highest = payload_slider
        filt = (spacex_df['Payload Mass (kg)'] > lowest) & (spacex_df['Payload Mass (kg)'] < highest)
        fig = px.scatter(
            spacex_df[filt], x="Payload Mass (kg)", y="class",
            color="Booster Version",
            size='Payload Mass (kg)')
    else:
        lowest, highest = payload_slider
        filtered_df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        filt = (filtered_df['Payload Mass (kg)'] > lowest) & (filtered_df['Payload Mass (kg)'] < highest)
        fig = px.scatter(
            filtered_df[filt], x="Payload Mass (kg)", y="class",
            color="Booster Version",
            size='Payload Mass (kg)')
    return fig



if __name__ == '__main__':
    app.run_server(debug=False, port=8052)